

# 1. Importación de Librerías


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# from google.colab import drive

# 2. Carga de Datos

In [19]:
# Fijate bien la ruta de los archivos.
df_hogar = pd.read_csv('../data/raw/EPH_usu_hogar_tic_t418.txt', delimiter=';')
df_personas = pd.read_csv('../data/raw/EPH_usu_Indiv_tic_t418.txt', delimiter=';')

# 3. Vista previa de los datos

In [20]:
print("Datos de Hogares:")
print(df_hogar.head())

print("\nDatos de Personas:")
print(df_personas.head())

Datos de Hogares:
                          CODUSU  NRO_HOGAR  REALIZADA  ANO4  TRIMESTRE  \
0  TQRMNOPPQHJMKMCDEFIAH00622969          1          1  2018          4   
1  TQRMNOPPQHJMLUCDEFIAH00622993          1          1  2018          4   
2  TQRMNOPPQHMMLUCDEFIAH00594749          1          1  2018          4   
3  TQRMNOPPRHJMLUCDEFIAH00623008          1          1  2018          4   
4  TQRMNOPPSHJMKMCDEFIAH00623022          1          1  2018          4   

   AGLOMERADO  POND_TIC  IH_II_01  IH_II_02  
0           2       539         1         1  
1           2       545         1         1  
2           2       612         1         1  
3           2       545         1         1  
4           2       539         1         1  

Datos de Personas:
                          CODUSU  NRO_HOGAR  COMPONENTE  ANO4  TRIMESTRE  \
0  TQRMNOPPQHJMKMCDEFIAH00622969          1           1  2018          4   
1  TQRMNOPPQHJMKMCDEFIAH00622969          1           2  2018          4   
2  TQRM

# 4. Limpieza y Preparación de Datos

In [21]:
# Selección de columnas relevantes para el análisis de hogares
hogar_cols = ['CODUSU', 'NRO_HOGAR', 'IH_II_01', 'IH_II_02']
df_hogar = df_hogar[hogar_cols]

# Renombrando columnas para mayor claridad
df_hogar.columns = ['codigo_vivienda', 'nro_hogar', 'tiene_computadora', 'acceso_internet']

# Eliminación de filas con valores faltantes
df_hogar = df_hogar.dropna()

# Selección de variables relevantes en datos de personas (añadir según necesidad)
personas_cols = ['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'ANO4', 'TRIMESTRE', 'IP_III_04', 'IP_III_05', 'IP_III_06']
df_personas = df_personas[personas_cols]

# Renombrando columnas para mayor claridad
df_personas.columns = ['codigo_vivienda', 'nro_hogar', 'componente', 'ano', 'trimestre', 'uso_internet', 'uso_computadora', 'uso_movil']

# Eliminación de filas con valores faltantes
df_personas = df_personas.dropna()


# 5. Análisis Descriptivo

In [22]:
# Proporción de hogares con acceso a internet
print("\nProporción de hogares con acceso a internet:")
print(df_hogar['acceso_internet'].value_counts(normalize=True))

# Proporción de uso de internet entre personas
print("\nProporción de personas que usan internet:")
print(df_personas['uso_internet'].value_counts(normalize=True))



Proporción de hogares con acceso a internet:
acceso_internet
1    0.816559
2    0.183065
9    0.000376
Name: proportion, dtype: float64

Proporción de personas que usan internet:
uso_internet
1    0.783252
2    0.216379
9    0.000369
Name: proportion, dtype: float64


# 6. Construcción del Modelo de Regresión Logística


## a. Unión de Datos de Hogar y Personas

In [23]:
# Unir datos de hogar y personas
df_hogar_personas = pd.merge(df_hogar, df_personas, on=['codigo_vivienda', 'nro_hogar'])

# Variables independientes y dependientes
X = df_hogar_personas[['tiene_computadora', 'uso_computadora', 'uso_movil']]
y = df_hogar_personas['acceso_internet']

# Convertir variables categóricas a numéricas si es necesario
X = pd.get_dummies(X, drop_first=True)

In [24]:
# Se guarga una copia del dataset de personas ya procesada en la carpeta processed.
df_hogar_personas.to_excel("../data/processed/EPH_usu_hogar_tic_t418_V2.xlsx")

## b. División de Datos en Conjuntos de Entrenamiento y Prueba


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## c. Entrenamiento del Modelo

In [26]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

## d. Evaluación del Modelo

In [27]:
y_pred = model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.861880152577827

Confusion Matrix:
[[13261   645     1]
 [ 1595   747     0]
 [    1     3     1]]

Classification Report:
              precision    recall  f1-score   support

           1       0.89      0.95      0.92     13907
           2       0.54      0.32      0.40      2342
           9       0.50      0.20      0.29         5

    accuracy                           0.86     16254
   macro avg       0.64      0.49      0.54     16254
weighted avg       0.84      0.86      0.85     16254

